In [126]:
const_vote[const_vote["PartyID"] == 705]

,ProvID,ConsID,PartyID,ApplyNo,Votes
1,10,201,705,8,0
16,10,202,705,5,77693
31,10,203,705,5,46
49,10,204,705,11,37298
63,10,205,705,5,0
...,...,...,...,...,...
4779,96,371,705,8,48906
4789,96,372,705,2,39360
4797,96,373,705,7,8
4806,96,374,705,1,1449


In [57]:
import pandas as pd
import numpy as np
import random
import difflib
import json
from datetime import datetime
from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)

party_list_score = pd.read_excel("Election66_Dev/ข้อมูลทำสื่อ-คะแนนพรรค.xlsx", header = [1])
const_score = pd.read_excel("Election66_Dev/ข้อมูลทำสื่อ.xlsx", header = [1])
party_list = pd.read_excel("Election66_Dev/บัญชีรายชื่อ/ข้อมูลบัญชีรายชื่อ.xlsx")
party = pd.read_excel("Election66_Dev/พรรคการเมือง.xlsx", sheet_name = "dbo_tblParty", header = [1])
party_color = pd.read_excel("Election66_Dev/พรรคการเมือง.xlsx", sheet_name = "62_code_colour")
party_abbrv = pd.read_excel("Election66_Dev/พรรคการเมือง.xlsx", sheet_name = "รายชื่อพรรค (ตัวย่อ)")
const_list = pd.read_excel("Election66_Dev/รายชื่อผู้สมัครแบบแบ่งเขต.xlsx")
const_district = pd.read_csv("Election66_Dev/Constituencies_district.csv")
const_district_1 = pd.read_excel("Election66_Dev/เขตเลือกตั้ง.xlsx", header = [1])
changwat_json = pd.read_json("Election66_Dev/จังหวัด_for_json.json")
kwang_bkk = pd.read_csv("Election66_Dev/แขวง กทม..csv", header = [1])

# Randomization Process
const_all = const_score[[
    "ProvID", "ConsID",
    "TotalVoter1", "VoterAppear1",
    "GoodBallot1", "BadBallot1",
    "NocastBallot1", "TotalUnit",
    "UnitFinish"]].drop_duplicates(subset = ["ProvID", "ConsID"]).reset_index(drop = True)

party_score_all = party_list_score[[
    "ProvID", "ConsID",
    "TotalVoter2", "VoterAppear2",
    "GoodBallot2", "BadBallot2",
    "NocastBallot2"]].drop_duplicates(subset = ["ProvID", "ConsID"]).reset_index(drop = True)

total_voters = 52241808
with open("./ref_json/province_ref_json.json", encoding = "utf-8-sig") as f:
    prov = json.loads(f.read())

prov_df = pd.DataFrame(prov)
prov_df["province_id"] = prov_df["province_id"].astype(int)

cons_json = pd.DataFrame(json.load(open("./info_constituency.json")))
const_all = const_all.merge(pd.read_excel("constituencies.xlsx", sheet_name = "prov_constituencies")\
            [["ConsID", "cons_no"]]).merge(prov_df[["province_id", "abbre_eng"]], left_on = ["ProvID"], right_on = ["province_id"])\
        .merge(cons_json[["prov_id", "cons_no", "total_vote_stations", "registered_vote"]], left_on = ["abbre_eng", "cons_no"], right_on = ["prov_id", "cons_no"])\
        .drop(columns = ["TotalUnit", "TotalVoter1"])\
        .rename(columns = {"total_vote_stations": "TotalUnit", "registered_vote": "TotalVoter1"})

party_score_all["TotalVoter2"] = const_all["TotalVoter1"]
party_score_all["VoterAppear2"] = const_all["VoterAppear1"] = const_all["TotalVoter1"].apply(
    lambda x: int(np.round(min(1, max(0, np.random.randn() * 0.1 + 0.75)) * x))
    )
const_all["BadBallot1"] = const_all["TotalVoter1"].apply(
    lambda x: int(np.round(min(1, max(0, np.random.randn() * 0.02 + 0.05)) * x))
    )
const_all["NocastBallot1"] = const_all["TotalVoter1"].apply(
    lambda x: int(np.round(min(1, max(0, np.random.randn() * 0.01 + 0.02)) * x))
    )
const_all["GoodBallot1"] = const_all.apply(
    lambda x: int(x.VoterAppear1 - x.BadBallot1 - x.NocastBallot1), axis = 1
)
party_score_all["BadBallot2"] = party_score_all["TotalVoter2"].apply(
    lambda x: int(np.round(min(1, max(0, np.random.randn() * 0.02 + 0.05)) * x))
    )
party_score_all["NocastBallot2"] = party_score_all["TotalVoter2"].apply(
    lambda x: int(np.round(min(1, max(0, np.random.randn() * 0.01 + 0.02)) * x))
    )
party_score_all["GoodBallot2"] = party_score_all.apply(
    lambda x: int(x.VoterAppear2 - x.BadBallot2 - x.NocastBallot2), axis = 1
)
party_score_all["TotalUnit"] = const_all["TotalUnit"]
party_score_all["UnitFinish"] = const_all["UnitFinish"] = const_all["TotalUnit"].apply(lambda x: int(round(x * 0.95, 0)))

def generate_vote(candidates, total):
    votes_list = []
    remain = total
    party_first = [705, 726, 709]
    for i in range(len(candidates) - 1):
        vote = random.randint(remain // 5 if candidates.reset_index()["PartyID"][i] in party_first else 0, remain // 3 if candidates.reset_index()["PartyID"][i] in party_first else 0)
        remain = remain - vote
        votes_list.append(vote)
    votes_list.append(total - sum(votes_list))
    return votes_list

def generate_all_const(const_vote, const_all, party = False):
    const_vote_final = pd.DataFrame()
    ballot = "2" if party else "1"
    vote = "PartyVote" if party else "Votes"
    for constituency in const_all[["ProvID", "ConsID"]].iterrows():
        tmp = const_vote[(const_vote.ProvID == constituency[1]["ProvID"]) & (const_vote.ConsID == constituency[1]["ConsID"])]
        total = const_all[(const_all.ProvID == constituency[1]["ProvID"]) & (const_all.ConsID == constituency[1]["ConsID"])]["GoodBallot" + ballot].tolist()[0]
        tmp[vote] = generate_vote(tmp, total)
        const_vote_final = pd.concat([const_vote_final, tmp])
    return const_vote_final

const_vote = const_score[["ProvID", "ConsID", "PartyID", "ApplyNo", "Votes"]]
const_vote = generate_all_const(const_vote, const_all)

party_list_vote = party_list_score[["ProvID", "ConsID", "PartyNumber", "PartyID", "PartyVote"]]
party_list_vote = generate_all_const(party_list_vote, party_score_all, party = True)

# data wrangling

party = party[["PartyID", "PartyNumber", "PartyName"]]\
    .merge(party_color.drop(columns = "count"), left_on = "PartyName", right_on = "name", how = "left")\
    .merge(party_abbrv[["พรรค", "อักษรย่อ"]], left_on = "PartyName", right_on = "พรรค", how = "left")

party = party.drop_duplicates(subset = "PartyID")

const_list = const_list[[
    "จังหวัด", "เขต", "หมายเลข", "คำนำหน้าชื่อ", "ชื่อ", "ชื่อกลาง", "ชื่อสกุล", "พรรค"
    ]]
const_list["ชื่อผู้สมัคร"] = const_list[["คำนำหน้าชื่อ", "ชื่อ", "ชื่อกลาง", "ชื่อสกุล"]].apply(
    lambda x: x["คำนำหน้าชื่อ"] + x["ชื่อ"] + ((" " + x["ชื่อกลาง"]) if type(x["ชื่อกลาง"]) == int else "") + " " + x["ชื่อสกุล"], axis = 1)
const_list["sorting"] = const_list[["ชื่อ", "ชื่อกลาง", "ชื่อสกุล"]].apply(
    lambda x: x["ชื่อ"] + ((" " + x["ชื่อกลาง"]) if type(x["ชื่อกลาง"]) == int else "") + " " + x["ชื่อสกุล"], axis = 1)

party_list = party_list.rename(columns = {"ชื่อ ชื่อกลาง นามสกุล": "ชื่อผู้สมัคร"})

const_list = const_list[["จังหวัด", "เขต", "หมายเลข", "พรรค", "ชื่อผู้สมัคร", "sorting"]]

const_bkk = const_district[const_district["จังหวัด"] == "กรุงเทพมหานคร"]
const_bkk["อำเภอ"] =\
    const_district[const_district["จังหวัด"] == "กรุงเทพมหานคร"]["อำเภอ"].apply(
        lambda x: difflib.get_close_matches(x, kwang_bkk["ชื่อ"].drop_duplicates().to_list())[0]
    )
const_bkk["ตำบล"] =\
    const_district[const_district["จังหวัด"] == "กรุงเทพมหานคร"]["ตำบล"].apply(
        lambda x: difflib.get_close_matches(x, kwang_bkk["ชื่อ.1"].drop_duplicates().to_list())[0] if type(x) == str else x
    )

const_district = pd.concat([const_district[const_district["จังหวัด"] != "กรุงเทพมหานคร"], const_bkk])

const_district_final = const_district_1.merge(changwat_json, left_on = "ProvID", right_on = "province_id").merge(const_district, left_on = ["province", "ConsNo"], right_on = ["จังหวัด", "เขต"])
const_district_final[["province", "ConsNo", "abbre_thai", "abbre_eng", "อำเภอ", "ตำบล", "flag", "optional"]]
const_district_final = const_district_final.rename(columns = {
    "ConsNo": "cons_no",
    "อำเภอ": "district",
    "ตำบล": "subdistrict",
})
const_final = const_district_final[["ProvID", "ConsID", "province", "cons_no", "abbre_thai", "abbre_eng", "eng", "district", "subdistrict", "flag", "optional"]]

# json etl
const_prov = const_final[["ProvID", "ConsID", "province", "cons_no", "abbre_eng"]].drop_duplicates()

const_final["final"] = const_final.apply(lambda x: (
                                        ((("แขวง" + x.subdistrict) if (x.province == "กรุงเทพมหานคร") else ("ตำบล" +
                                        x.subdistrict) if (x.subdistrict[:6] != "เทศบาล") else (x.subdistrict)) if type(x.subdistrict) == str else "") +
                                        ((" (" + (("แขวง" + x.optional) if (x.province == "กรุงเทพมหานคร") else ("ตำบล" +
                                        x.optional) if (x.subdistrict[:6] == "เทศบาล") else (x.optional)) + ")") if type(x.optional) == str else "")).strip(), axis = 1)
const_final = const_final.groupby(["province", "cons_no", "abbre_thai", "abbre_eng", "district", "flag"], dropna = False)["final"].apply(list).reset_index()
const_final["zone"] = const_final.apply(
    lambda x: (("เขต" if x.province == "กรุงเทพมหานคร" else "อำเภอ") + (x.district) +\
    ((" (" + (x.flag if type(x.flag) == str else "") + (", ".join(x["final"])) + ")") if (type(x.flag) == str) else "") if type(x.district) == str else ""), axis = 1
)

const_final = const_final[["province", "cons_no", "abbre_thai", "abbre_eng","zone"]]

# with pd.ExcelWriter("constituencies.xlsx", mode = "a") as writer:
#     const_final.to_excel(writer, sheet_name = "constituencies", index = False)
#     # const_prov.to_excel(writer, sheet_name = "prov_constituencies", index = False)
#     const_list.to_excel(writer, sheet_name = "candidate_constituencies", index = False)
# with pd.ExcelWriter("party.xlsx") as writer:
#     party.drop(columns = ["พรรค", "name"])\
#         .to_excel(writer, sheet_name = "party_overview", index = False)
#     party_list.to_excel(writer, sheet_name = "party_list", index = False)

In [145]:
const_vote.sort_values("Votes", ascending = False)

,ProvID,ConsID,PartyID,ApplyNo,Votes
1248,27,536,774,5,77963
3496,62,249,775,8,73464
877,17,541,769,2,58614
14,10,201,769,2,57499
4073,77,401,769,4,55554
...,...,...,...,...,...
2556,44,447,707,15,0
889,18,281,701,5,0
2558,44,447,719,4,0
887,18,280,763,9,0


In [3]:
party_list_vote

,ProvID,ConsID,PartyNumber,PartyID,PartyVote
0,10,201,1,778,0
1,10,201,2,712,532
2,10,201,3,747,0
3,10,201,4,776,0
4,10,201,5,761,1445
...,...,...,...,...,...
26795,96,375,63,702,0
26796,96,375,64,729,0
26797,96,375,65,777,0
26798,96,375,66,722,30


In [39]:
const_final

,province,cons_no,abbre_thai,abbre_eng,zone
0,กระบี่,1,กบ,KBI,"[อำเภอเมืองกระบี่, อำเภอเหนือคลอง (ยกเว้นตำบลโ..."
1,กระบี่,2,กบ,KBI,"[อำเภอปลายพระยา, อำเภออ่าวลึก, อำเภอเขาพนม]"
2,กระบี่,3,กบ,KBI,"[อำเภอคลองท่อม, อำเภอลำทับ, อำเภอเกาะลันตา, อำ..."
3,กรุงเทพมหานคร,1,กทม,BKK,"[เขตดุสิต (ยกเว้นแขวงถนนนครไชยศรี), เขตบางรัก,..."
4,กรุงเทพมหานคร,2,กทม,BKK,"[เขตปทุมวัน, เขตราชเทวี, เขตสาทร]"
...,...,...,...,...,...
395,แพร่,1,พร,PRE,"[อำเภอสูงเม่น (ยกเว้นตำบลหัวฝาย, ตำบลบ้านปง), ..."
396,แพร่,2,พร,PRE,"[อำเภอร้องกวาง, อำเภอสอง, อำเภอหนองม่วงไข่, อำ..."
397,แพร่,3,พร,PRE,"[อำเภอลอง, อำเภอวังชิ้น, อำเภอสูงเม่น (เฉพาะตำ..."
398,แม่ฮ่องสอน,1,มส,MSN,"[อำเภอขุนยวม, อำเภอปางมะผ้า, อำเภอปาย, อำเภอเม..."


# Overall

In [56]:
overview = dict()
overview["status"] = True

tmp_0 = party_score_all.rename(columns = {
    "TotalVoter2": "registered_votes",
    "VoterAppear2": "turn_out",
    "GoodBallot2": "valid_votes_party_list",
    "TotalUnit": "total_vote_stations",
    "UnitFinish": "counted_vote_stations"
}).merge(const_prov, on = ["ProvID", "ConsID"], how = "left")

overview.update(json.loads(tmp_0.sum()[["registered_votes", "turn_out", "valid_votes_party_list", "total_vote_stations", "counted_vote_stations"]].to_json()))

tmp_2 = const_vote.merge(const_prov, on = ["ProvID", "ConsID"], how = "left")\
    .merge(const_list, left_on = ["province", "cons_no", "ApplyNo"], right_on = ["จังหวัด", "เขต", "หมายเลข"], how = "right")\
    .merge(party, on = "PartyID", how = "left")[["province", "cons_no", "หมายเลข", "ชื่อผู้สมัคร", "PartyName", "color", "hilightColor", "อักษรย่อ", "Votes"]]\
    .rename(columns = {
        "หมายเลข": "apply_no",
        "ชื่อผู้สมัคร": "candidate_name",
        "PartyName": "party_name",
        "hilightColor": "highlight_color",
        "อักษรย่อ": "party_abbrv",
        "Votes": "votes"
    })

tmp_3 = party_list_vote.merge(const_prov, on = ["ProvID", "ConsID"])\
            .merge(party).rename(columns = {
                                            "PartyName": "party_name",
                                            "PartyNumber": "party_no",
                                            "อักษรย่อ": "party_abbrv",
                                            "hilightColor": "highlight_color",
                                            "PartyVote": "party_vote"
                                        })

# overview["valid_votes_constituency"] = int(tmp_2.votes.sum())
overview["percent_turn_out"] = round(float(overview["turn_out"]) / float(overview["registered_votes"]) * 100, 5)
overview["percent_count"] = round(float(overview["counted_vote_stations"]) / float(overview["total_vote_stations"]) * 100, 5)

overview["result_party"] = json.loads(tmp_3.groupby(["party_name", "party_no", "party_abbrv", "color", "highlight_color"], dropna = False).sum()["party_vote"]\
                                        .reset_index().to_json(orient = "records"))

for party_overview in overview["result_party"]:
    party_overview["pm_candidates"] = [{
            "pm_cand_name": "นาย ทดสอบ สามารถ",
            "pic_pm_cand": "https://cdn-gbdi.ectreport.com/pm-candidate/{}-1.png".format(party_overview["party_name"])
        }]

overview["result_const"] = json.loads(const_final[["province", "abbre_eng"]].drop_duplicates().to_json(orient = "records"))

for prov_overview in overview["result_const"]:
    prov_overview["registered_votes"] = int(tmp_0[tmp_0["province"] == prov_overview["province"]]["registered_votes"].sum())
    prov_overview["turn_out"] = int(tmp_0[tmp_0["province"] == prov_overview["province"]]["turn_out"].sum())
    # prov_overview["valid_votes_party_list"] = int(tmp_0[tmp_0["province"] == prov_overview["province"]]["valid_votes_party_list"].sum())
    # prov_overview["valid_votes_constituency"] = int(tmp_2[tmp_2["province"] == prov_overview["province"]]["votes"].sum())
    prov_overview["total_vote_stations"] = int(tmp_0[tmp_0["province"] == prov_overview["province"]]["total_vote_stations"].sum())
    prov_overview["counted_vote_stations"] = int(tmp_0[tmp_0["province"] == prov_overview["province"]]["counted_vote_stations"].sum())
    prov_overview["percent_turn_out"] = round(prov_overview["turn_out"] / prov_overview["registered_votes"] * 100, 5) 
    prov_overview["percent_count"] = round(prov_overview["counted_vote_stations"] / prov_overview["total_vote_stations"] * 100, 5)
    prov_overview["party_list"] = json.loads(tmp_3[tmp_3["province"] == prov_overview["province"]]\
                                    .drop(columns = ["province", "cons_no"]).drop(columns = ["ProvID", "ConsID", "PartyID", "abbre_eng", "name", "พรรค"])\
                                    .groupby(["party_no", "party_name", "color", "highlight_color", "party_abbrv"], dropna = False).sum().reset_index()\
                                    .to_json(orient = "records"))
    search_first_prov = tmp_2[(tmp_2["province"] == prov_overview["province"])].groupby(["province", "cons_no"]).first().reset_index().groupby("party_name").count()[["votes"]].reset_index()

    for party_prov in prov_overview["party_list"]:
        party_prov["constituency_vote"] = int(tmp_2[(tmp_2["province"] == prov_overview["province"]) & (tmp_2["party_name"] == party_prov["party_name"])]["votes"].sum())
        party_prov["percent_party_list"] = round(party_prov["party_vote"] / prov_overview["registered_votes"] * 100, 5)
        party_prov["percent_constituency"] = round(party_prov["constituency_vote"] / prov_overview["registered_votes"] * 100, 5)
        party_prov["pic_party"] = "https://cdn-gbdi.ectreport.com/party/{}.png".format(party_prov["party_name"])
        gainedProv = search_first_prov[search_first_prov["party_name"] == party_prov["party_name"]]["votes"].to_list()
        party_prov["gained_seats"] = gainedProv[0] if gainedProv != [] else 0

for district in tqdm(const_final[["province", "cons_no"]].drop_duplicates().iterrows()):
    tmp = const_final[
        (const_final["province"] == district[1]["province"]) & 
        (const_final["cons_no"] == district[1]["cons_no"])
    ]
    
    tmp_dis_2 = tmp_2[
        (tmp_2["province"] == district[1]["province"]) & 
        (tmp_2["cons_no"] == district[1]["cons_no"])
    ]

    tmp_dis_3 = tmp_3[
        (tmp_3["province"] == district[1]["province"])
    ]

    for prov in overview["result_const"]:
        if prov["province"] == district[1]["province"]:
            final = json.loads(tmp.groupby("cons_no")["zone"].apply(list).reset_index().to_json(orient = "records"))[0]
            final["candidates"] = json.loads(tmp_dis_2.drop(columns = ["province", "cons_no"]).to_json(orient = "records"))
            try: prov["constituencies"].append(final)
            except: prov["constituencies"] = [final]

            for const in prov["constituencies"]:
                # const["valid_votes_constituency"] = int(tmp_2[(tmp_2["province"] == district[1]["province"]) & (tmp_2["cons_no"] == const["cons_no"])]["votes"].sum())
                const["registered_votes"] = int(tmp_0[(tmp_0["province"] == district[1]["province"]) & (tmp_0["cons_no"] == const["cons_no"])]["registered_votes"].sum())
                const["turn_out"] = int(tmp_0[(tmp_0["province"] == district[1]["province"]) & (tmp_0["cons_no"] == const["cons_no"])]["turn_out"].sum())
                for cand_const in const["candidates"]:
                    cand_const["percent_constituency"] = round(cand_const["votes"] / const["registered_votes"] * 100, 5)
                    cand_const["pic_candidates"] = "https://cdn-gbdi.ectreport.com/candidates/{}-{}-{}.png".format(prov["abbre_eng"], const["cons_no"], cand_const["apply_no"])
                    cand_const["pic_party"] = "https://cdn-gbdi.ectreport.com/party/{}.png".format(cand_const["party_name"])

tmp_2_party = tmp_2.sort_values("votes", ascending = False)\
    .groupby(["province", "cons_no"]).first().reset_index()\
    .groupby("party_name").count().reset_index()[["party_name", "votes"]]

for party_current in overview["result_party"]:
    result_seats = tmp_2_party[tmp_2_party["party_name"] == party_current["party_name"]]["votes"].to_list()
    result_const_vote = tmp_2.groupby("party_name").sum().reset_index()
    result_const_vote = result_const_vote[result_const_vote["party_name"] == party_current["party_name"]]["votes"].to_list()
    party_current["gained_seats"] = result_seats[0] if result_seats != [] else 0
    party_current["constituency_vote"] = result_const_vote[0]
    party_current["percent_party_list"] = round(float(party_current["party_vote"]) / float(overview["registered_votes"]) * 100, 5)
    party_current["percent_constituency"] = round(float(party_current["constituency_vote"]) / float(overview["registered_votes"]) * 100, 5)
    party_current["pic_party"] = "https://cdn-gbdi.ectreport.com/party/{}.png".format(party_current["party_name"])
    party_current["party_list"] = json.loads(party_list[party_list["พรรค"] == party_current["party_name"]]\
                                            .rename(columns = {"ลำดับที่": "list_no", "ชื่อผู้สมัคร": "party_list_name"}).drop(columns = "พรรค").to_json(orient = "records"))
    for party_list_cand in party_current["party_list"]:
        party_list_cand["pic_party_list"] = "https://cdn-gbdi.ectreport.com/party-list/{}-{}.png".format(party_current["party_name"], party_list_cand["list_no"])
    

overview["last_update"] = datetime.now().strftime("%Y-%m-%d %X GMT+0700")
overview["author"] = "Government Big Data Institute"

with open("overview.json", mode = "w") as f:
    f.write(json.dumps(overview, ensure_ascii = False).encode('utf8').decode())

NameError: name 'party_score_all' is not defined

# Stats and info

In [3]:
const_prov

,ProvID,ConsID,province,cons_no,abbre_eng
0,10,201,กรุงเทพมหานคร,1,BKK
5,10,202,กรุงเทพมหานคร,2,BKK
8,10,203,กรุงเทพมหานคร,3,BKK
10,10,204,กรุงเทพมหานคร,4,BKK
12,10,205,กรุงเทพมหานคร,5,BKK
...,...,...,...,...,...
2174,96,371,นราธิวาส,1,NWT
2176,96,372,นราธิวาส,2,NWT
2178,96,373,นราธิวาส,3,NWT
2181,96,374,นราธิวาส,4,NWT


# รันเซลล์ในนี้

In [63]:
import pandas as pd
import json

regis_21_apr = pd.read_excel("./230512_consituencies.xlsx", sheet_name = "adjust_Data หน่วย ส.ส. 66 Final")
regis_21_apr = regis_21_apr.dropna(how = "any")
regis_21_apr["จำนวนหน่วยเลือกตั้ง"] += 1
regis_21_apr 

,ลำดับ,จังหวัด,เขตเลือกตั้ง,จำนวนผู้มีสิทธิเลือกตั้ง,จำนวนหน่วยเลือกตั้ง
0,1.0,กรุงเทพมหานคร,1.0,137544,239
1,2.0,กรุงเทพมหานคร,2.0,140711,229
2,3.0,กรุงเทพมหานคร,3.0,125506,214
3,4.0,กรุงเทพมหานคร,4.0,130229,194
4,5.0,กรุงเทพมหานคร,5.0,132325,204
...,...,...,...,...,...
395,396.0,อุบลราชธานี,7.0,128533,268
396,397.0,อุบลราชธานี,8.0,136022,289
397,398.0,อุบลราชธานี,9.0,128034,225
398,399.0,อุบลราชธานี,10.0,139504,302


In [59]:
regis_const = pd.read_excel("./constituencies.xlsx", sheet_name = "prov_constituencies")
regis_const = regis_const.merge(regis_21_apr, left_on = ["province", "cons_no"], right_on = ["จังหวัด", "เขตเลือกตั้ง"])\
    .drop(columns = ["total_vote_stations", "registered_vote", "จังหวัด", "เขตเลือกตั้ง","ลำดับ"])\
    .rename(columns = {
        "จำนวนหน่วยเลือกตั้ง": "total_vote_stations",
        "จำนวนผู้มีสิทธิเลือกตั้ง": "registered_vote"})
regis_const[["total_vote_stations", "registered_vote"]] = regis_const[["total_vote_stations", "registered_vote"]].astype(int)
final = regis_const.merge(const_final.groupby(["province", "cons_no"])["zone"].apply(list).reset_index()).drop(columns = ["ProvID", "ConsID", "province"])
final["cons_id"] = final["abbre_eng"] + "_" + final["cons_no"].astype(str)
final = final.rename(columns = {"abbre_eng": "prov_id"})[["cons_id", "cons_no", "prov_id", "zone", "total_vote_stations", "registered_vote"]].sort_values("cons_id")

In [60]:
final_province = final.groupby("prov_id").sum().reset_index().merge(pd.DataFrame(prov), left_on = "prov_id", right_on = "abbre_eng")\
    [["province_id", "prov_id", "province", "abbre_thai", "eng", "total_vote_stations", "registered_vote"]]\
    .rename(columns = {"registered_vote": "total_registered_vote"})

from pythainlp.util import collate
list2 = collate(final_province["province"].tolist())
idx = {x:i for i,x in enumerate(final_province["province"])}
final_province = final_province.reindex([idx[x] for x in list2])
final_province

,province_id,prov_id,province,abbre_thai,eng,total_vote_stations,total_registered_vote
14,81,KBI,กระบี่,กบ,KRABI,587,357245
3,10,BKK,กรุงเทพมหานคร,กทม,BANGKOK,6360,4479155
17,71,KRI,กาญจนบุรี,กจ,KANCHANABURI,1083,646081
18,46,KSN,กาฬสินธุ์,กส,KALASIN,1617,788214
16,62,KPT,กำแพงเพชร,กพ,KAMPHAENG PHET,1123,567999
...,...,...,...,...,...,...,...
0,37,ACR,อำนาจเจริญ,อจ,AMNAT CHAROEN,641,301354
72,41,UDN,อุดรธานี,อด,UDON THANI,2252,1248824
74,53,UTT,อุตรดิตถ์,อต,UTTARADIT,732,368323
73,61,UTI,อุทัยธานี,อน,UTHAI THANI,617,260983


In [61]:
json_province = {"total_registered_vote": int(final["registered_vote"].sum()), "total_vote_stations": int(final["total_vote_stations"].sum())}
json_province["province"] = json.loads(final_province.to_json(orient = "records", force_ascii = False))
with open("./info_province.json", mode = "w") as f:
    f.write(json.dumps(json_province, ensure_ascii = False))
json_province

{'total_registered_vote': 52238594,
 'total_vote_stations': 95175,
 'province': [{'province_id': '81',
   'prov_id': 'KBI',
   'province': 'กระบี่',
   'abbre_thai': 'กบ',
   'eng': 'KRABI',
   'total_vote_stations': 587,
   'total_registered_vote': 357245},
  {'province_id': '10',
   'prov_id': 'BKK',
   'province': 'กรุงเทพมหานคร',
   'abbre_thai': 'กทม',
   'eng': 'BANGKOK',
   'total_vote_stations': 6360,
   'total_registered_vote': 4479155},
  {'province_id': '71',
   'prov_id': 'KRI',
   'province': 'กาญจนบุรี',
   'abbre_thai': 'กจ',
   'eng': 'KANCHANABURI',
   'total_vote_stations': 1083,
   'total_registered_vote': 646081},
  {'province_id': '46',
   'prov_id': 'KSN',
   'province': 'กาฬสินธุ์',
   'abbre_thai': 'กส',
   'eng': 'KALASIN',
   'total_vote_stations': 1617,
   'total_registered_vote': 788214},
  {'province_id': '62',
   'prov_id': 'KPT',
   'province': 'กำแพงเพชร',
   'abbre_thai': 'กพ',
   'eng': 'KAMPHAENG PHET',
   'total_vote_stations': 1123,
   'total_regist

In [62]:
# import json
# with open("./ref_json/province_ref_json.json", encoding = "utf-8-sig") as f:
#     provinces = json.loads(f.read())

# final = {"total_vote_stations": int(regis_const["total_vote_stations"].sum()), "total_registered_vote": int(regis_const["registered_vote"].sum()), "province": []}
# for province in provinces:
#     province["total_vote_stations"] = int(regis_const[regis_const["province"] == province["province"]]["total_vote_stations"].sum())
#     province["total_registered_vote"] = int(regis_const[regis_const["province"] == province["province"]]["registered_vote"].sum())
#     final["province"].append(province)
with open("./info_constituency.json", mode = "w") as f:
    f.write(json.dumps(json.loads(final.sort_values(["prov_id", "cons_no"]).to_json(orient = "records", force_ascii = False)), ensure_ascii = False))

In [70]:
import json
with open("pm_candidate.json") as f:
    a = json.loads(f.read())
a

[{'party_name': 'ก้าวไกล',
  'pm_candidates': [{'name': 'นายพิธา ลิ้มเจริญรัตน์'}]},
 {'party_name': 'คลองไทย',
  'pm_candidates': [{'name': 'นายสายัณห์ อินทรภักดิ์'}]},
 {'party_name': 'ความหวังใหม่',
  'pm_candidates': [{'name': 'นายชิงชัย มงคลธรรม'}]},
 {'party_name': 'ช่วยชาติ',
  'pm_candidates': [{'name': 'นางสาวนงนุช บัวใหญ่'}]},
 {'party_name': 'ชาติไทยพัฒนา',
  'pm_candidates': [{'name': 'นายวราวุธ ศิลปอาชา'}]},
 {'party_name': 'ชาติพัฒนากล้า',
  'pm_candidates': [{'name': 'นายสุวัจน์ ลิปตพัลลภ'},
   {'name': 'นายกรณ์ จาติกวณิช'},
   {'name': 'นายเทวัญ ลิปตพัลลภ'}]},
 {'party_name': 'ชาติรุ่งเรือง',
  'pm_candidates': [{'name': 'นางพิมไหมทอง ศักดิพัตโภคิน'}]},
 {'party_name': 'ท้องที่ไทย',
  'pm_candidates': [{'name': 'นายบัญชา เดชเจริญศิริกุล'}]},
 {'party_name': 'ทางเลือกใหม่',
  'pm_candidates': [{'name': 'นายราเชน ตระกูลเวียง'}]},
 {'party_name': 'ไทยก้าวหน้า',
  'pm_candidates': [{'name': 'นายวัชรพล บุษมงคล'},
   {'name': 'พลเอก สิทธิ์ สิทธิมงคล'}]},
 {'party_name': 'ไทยช

In [64]:
party_list_cand = party_list\
    .merge(party[["PartyNumber", "PartyName"]], left_on = "พรรค", right_on = "PartyName")\
    .drop(columns = ["PartyName", "พรรค"])\
    .rename(columns = {"PartyNumber": "party_no", "ลำดับที่": "list_no", "ชื่อผู้สมัคร": "name"})
party_list_cand["party_list_candidates"] = party_list_cand[["list_no", "name"]].apply(lambda x: dict([("list_no", x.list_no), ("name", x["name"]), ("image_url", "https://ectdatastorage.blob.core.windows.net/images/candidates/" + x["name"].replace(" ","_") + ".png")]), axis = 1)
final_json = party_list_cand.drop(columns = ["list_no", "name"]).groupby("party_no")["party_list_candidates"].apply(list).reset_index()
with open("./info_party_candidate.json", mode = "w") as f:
    f.write(json.dumps(json.loads(final_json.to_json(orient = "records", force_ascii = False)), ensure_ascii = False))

In [7]:
party[party["PartyNumber"] == 26]["PartyNumber"].tolist()[0]

26

In [71]:
json_final = json.loads(final_json.to_json(orient = "records", force_ascii = False))
for i in a:
    for pn in json_final:
        if party[party["PartyName"] == i["party_name"]]["PartyNumber"].tolist()[0] == pn["party_no"]:
            pn["pm_candidates"] = i["pm_candidates"]
            for pm in pn["pm_candidates"]:
                pm["image_url"] = "https://ectdatastorage.blob.core.windows.net/images/pm_candidates/" + pm["name"].replace(" ","_") + ".png"
            break

with open("./info_party_candidate.json", mode = "w") as f:
    f.write(json.dumps(json_final, ensure_ascii = False))

In [5]:
with open("./ref_json/province_ref_json.json", encoding = "utf-8-sig") as f:
    prov = json.loads(f.read())
prov

[{'province_id': '81',
  'province': 'กระบี่',
  'abbre_thai': 'กบ',
  'abbre_eng': 'KBI',
  'eng': 'KRABI'},
 {'province_id': '10',
  'province': 'กรุงเทพมหานคร',
  'abbre_thai': 'กทม',
  'abbre_eng': 'BKK',
  'eng': 'BANGKOK'},
 {'province_id': '71',
  'province': 'กาญจนบุรี',
  'abbre_thai': 'กจ',
  'abbre_eng': 'KRI',
  'eng': 'KANCHANABURI'},
 {'province_id': '46',
  'province': 'กาฬสินธุ์',
  'abbre_thai': 'กส',
  'abbre_eng': 'KSN',
  'eng': 'KALASIN'},
 {'province_id': '62',
  'province': 'กำแพงเพชร',
  'abbre_thai': 'กพ',
  'abbre_eng': 'KPT',
  'eng': 'KAMPHAENG PHET'},
 {'province_id': '40',
  'province': 'ขอนแก่น',
  'abbre_thai': 'ขก',
  'abbre_eng': 'KKN',
  'eng': 'KHON KAEN'},
 {'province_id': '22',
  'province': 'จันทบุรี',
  'abbre_thai': 'จบ',
  'abbre_eng': 'CTI',
  'eng': 'CHANTHABURI'},
 {'province_id': '24',
  'province': 'ฉะเชิงเทรา',
  'abbre_thai': 'ฉช',
  'abbre_eng': 'CCO',
  'eng': 'CHACHOENGSAO'},
 {'province_id': '20',
  'province': 'ชลบุรี',
  'abbre_tha

In [20]:
const_json = pd.read_excel("constituencies.xlsx", sheet_name = "constituencies")\
    .groupby(["province", "cons_no", "abbre_eng"])["zone"]\
    .apply(list).reset_index().merge(pd.read_excel("constituencies.xlsx", sheet_name = "prov_constituencies"))
const_json["zone"] = const_json["zone"].apply(lambda x: [] if ((len(x) == 1) and (type(x[0]) != str)) else x)
const_json["cons_id"] = const_json["abbre_eng"].str.cat(const_json["cons_no"].astype(str), sep = "_")
const_json_export = json.loads(const_json[["cons_id", "cons_no", "zone", "total_vote_stations", "registered_vote"]].to_json(orient = "records", force_ascii = False))

final_const_json = []
for record in const_json_export:
    for prov_one in prov:
        if record["cons_id"].split("_")[0] == prov_one["abbre_eng"]:
            record["prov_id"] = prov_one["province_id"]
            final_const_json.append({
                "cons_id": record["cons_id"],
                "cons_no": record["cons_no"],
                "prov_id": record["cons_id"].split("_")[0],
                "zone": record["zone"],
                "total_vote_stations": record["total_vote_stations"],
                "registered_vote": record["registered_vote"]
            })
            break

with open("info_constituency.json", mode = "w") as f:
    f.write(json.dumps(final_const_json, ensure_ascii = False))
const_json

,province,cons_no,abbre_eng,zone,ProvID,ConsID,total_vote_stations,registered_vote,cons_id
0,กระบี่,1,KBI,"[อำเภอเมืองกระบี่, อำเภอเหนือคลอง (ยกเว้นตำบลโ...",81,234,185,122761,KBI_1
1,กระบี่,2,KBI,"[อำเภอปลายพระยา, อำเภออ่าวลึก, อำเภอเขาพนม]",81,235,188,114551,KBI_2
2,กระบี่,3,KBI,"[อำเภอคลองท่อม, อำเภอลำทับ, อำเภอเกาะลันตา, อำ...",81,236,211,119944,KBI_3
3,กรุงเทพมหานคร,1,BKK,"[เขตดุสิต (ยกเว้นแขวงถนนนครไชยศรี), เขตบางรัก,...",10,201,238,137615,BKK_1
4,กรุงเทพมหานคร,2,BKK,"[เขตปทุมวัน, เขตราชเทวี, เขตสาทร]",10,202,228,140757,BKK_2
...,...,...,...,...,...,...,...,...,...
395,แพร่,1,PRE,"[อำเภอสูงเม่น (ยกเว้นตำบลหัวฝาย, ตำบลบ้านปง), ...",54,439,245,123836,PRE_1
396,แพร่,2,PRE,"[อำเภอร้องกวาง, อำเภอสอง, อำเภอหนองม่วงไข่, อำ...",54,440,290,118559,PRE_2
397,แพร่,3,PRE,"[อำเภอลอง, อำเภอวังชิ้น, อำเภอสูงเม่น (เฉพาะตำ...",54,441,270,123643,PRE_3
398,แม่ฮ่องสอน,1,MSN,"[อำเภอขุนยวม, อำเภอปางมะผ้า, อำเภอปาย, อำเภอเม...",58,453,237,87277,MSN_1


In [48]:
prov_df = pd.DataFrame(prov)
prov_df

,province_id,province,abbre_thai,abbre_eng,eng
0,81,กระบี่,กบ,KBI,KRABI
1,10,กรุงเทพมหานคร,กทม,BKK,BANGKOK
2,71,กาญจนบุรี,กจ,KRI,KANCHANABURI
3,46,กาฬสินธุ์,กส,KSN,KALASIN
4,62,กำแพงเพชร,กพ,KPT,KAMPHAENG PHET
...,...,...,...,...,...
72,37,อำนาจเจริญ,อจ,ACR,AMNAT CHAROEN
73,41,อุดรธานี,อด,UDN,UDON THANI
74,53,อุตรดิตถ์,อต,UTT,UTTARADIT
75,61,อุทัยธานี,อน,UTI,UTHAI THANI


In [123]:
party_1 = pd.read_excel("./party_overview.xlsm", sheet_name = "party_overview")

In [124]:
const_list

,จังหวัด,เขต,หมายเลข,พรรค,ชื่อผู้สมัคร
3489,นครศรีธรรมราช,8,5,ไทยสร้างไทย,นางกชพร ราชรักษ์
2022,อุดรธานี,10,10,ไทยศรีวิไลย์,ว่าที่ร้อยตรีกตัญญู คันธี
2779,แม่ฮ่องสอน,1,1,เสรีรวมไทย,พันตำรวจเอกกนิษฐ ประสานสุข
905,ระยอง,1,1,ก้าวไกล,นางสาวกมนทรรศน์ กิตติสุนทรสกุล
1786,ขอนแก่น,1,13,ไทยภักดี,นายกมล กิจกสิวัฒน์
...,...,...,...,...,...
4181,นราธิวาส,4,9,พลังสังคมใหม่,นายฮาเซ็ง อิงดิง
3758,สงขลา,8,8,เพื่อไทย,นายฮานาพี หมีนเส็น
4164,สตูล,2,11,คลองไทย,นายฮาบีบุญเลาะ เตาวะโต
276,กรุงเทพมหานคร,21,8,เสรีรวมไทย,นายฮารูน เมฆลอย


In [140]:
from pythainlp.util import collate
list2 = collate(const_list["sorting"].tolist())
idx = {x:i for i,x in enumerate(const_list["sorting"])}
const_list = const_list.reindex([idx[x] for x in list2]).drop(columns = "sorting")
const_list

,จังหวัด,เขต,หมายเลข,พรรค,ชื่อผู้สมัคร
3489,นครศรีธรรมราช,8,5,ไทยสร้างไทย,นางกชพร ราชรักษ์
2022,อุดรธานี,10,10,ไทยศรีวิไลย์,ว่าที่ร้อยตรีกตัญญู คันธี
2779,แม่ฮ่องสอน,1,1,เสรีรวมไทย,พันตำรวจเอกกนิษฐ ประสานสุข
905,ระยอง,1,1,ก้าวไกล,นางสาวกมนทรรศน์ กิตติสุนทรสกุล
1786,ขอนแก่น,1,13,ไทยภักดี,นายกมล กิจกสิวัฒน์
...,...,...,...,...,...
4181,นราธิวาส,4,9,พลังสังคมใหม่,นายฮาเซ็ง อิงดิง
3758,สงขลา,8,8,เพื่อไทย,นายฮานาพี หมีนเส็น
4164,สตูล,2,11,คลองไทย,นายฮาบีบุญเลาะ เตาวะโต
276,กรุงเทพมหานคร,21,8,เสรีรวมไทย,นายฮารูน เมฆลอย


In [142]:
const_list = const_list.reset_index(drop = True)

In [145]:
const_json = const_list.reset_index().merge(party_1, left_on = "พรรค", right_on = "name", how = "left")\
    .merge(prov_df, left_on = "จังหวัด", right_on = "province").sort_values("index")\
    [["abbre_eng", "เขต", "หมายเลข", "id", "ชื่อผู้สมัคร"]]
const_json["mp_app_id"] = const_json["abbre_eng"] + "_" + const_json["เขต"].astype(str) + "_" + const_json["หมายเลข"].astype(str)
const_json["id"] = const_json["id"].astype(int)
const_json["image_url"] = "https://ectdatastorage.blob.core.windows.net/images/mp_candidates/" + const_json["ชื่อผู้สมัคร"].str.replace(" ","_") + ".png"

with open("info_mp_candidate.json", mode = "w") as f:
    f.write(json.dumps(json.loads(const_json[["mp_app_id", "หมายเลข", "id", "ชื่อผู้สมัคร", "image_url"]]\
        .rename(columns = {
            "หมายเลข": "mp_app_no",
            "id": "mp_app_party_id",
            "ชื่อผู้สมัคร": "mp_app_name"
        }).to_json(orient = "records", force_ascii = False)), ensure_ascii = False))

In [144]:
const_json

,abbre_eng,เขต,หมายเลข,id,ชื่อผู้สมัคร,mp_app_id,image_url
0,NST,8,5,762,นางกชพร ราชรักษ์,NST_8_5,https://ectdatastorage.blob.core.windows.net/i...
123,UDN,10,10,733,ว่าที่ร้อยตรีกตัญญู คันธี,UDN_10_10,https://ectdatastorage.blob.core.windows.net/i...
284,MSN,1,1,719,พันตำรวจเอกกนิษฐ ประสานสุข,MSN_1_1,https://ectdatastorage.blob.core.windows.net/i...
307,RYG,1,1,726,นางสาวกมนทรรศน์ กิตติสุนทรสกุล,RYG_1_1,https://ectdatastorage.blob.core.windows.net/i...
356,KKN,1,13,769,นายกมล กิจกสิวัฒน์,KKN_1_13,https://ectdatastorage.blob.core.windows.net/i...
...,...,...,...,...,...,...,...
1271,NWT,4,9,761,นายฮาเซ็ง อิงดิง,NWT_4_9,https://ectdatastorage.blob.core.windows.net/i...
3917,SKA,8,8,705,นายฮานาพี หมีนเส็น,SKA_8_8,https://ectdatastorage.blob.core.windows.net/i...
4467,STN,2,11,742,นายฮาบีบุญเลาะ เตาวะโต,STN_2_11,https://ectdatastorage.blob.core.windows.net/i...
1140,BKK,21,8,719,นายฮารูน เมฆลอย,BKK_21_8,https://ectdatastorage.blob.core.windows.net/i...


JSON composition

In [14]:
const_all

,ProvID,ConsID,TotalVoter1,VoterAppear1,GoodBallot1,BadBallot1,NocastBallot1,TotalUnit,UnitFinish
0,10,201,130934,86452,75924,10528,0,186,177
1,10,202,132090,108257,102366,4642,1249,249,237
2,10,203,131248,116260,110197,2925,3138,255,242
3,10,204,129060,110443,101295,5884,3264,224,213
4,10,205,130489,121634,109384,9610,2640,262,249
...,...,...,...,...,...,...,...,...,...
395,96,371,128684,128684,116787,7801,4096,249,237
396,96,372,130976,85962,77678,5132,3152,183,174
397,96,373,130613,66076,57063,7047,1966,186,177
398,96,374,129558,105551,98627,4450,2474,231,219


## พรรคการเมืองหาย

In [15]:
import pandas as pd
prov_ref = pd.read_json("ref_json/info_constituency.json")
const_data = const_all.merge(const_district_1)\
    .merge(
        prov_ref[["cons_id", "cons_no", "prov_id"]],
        left_on = ["ProvID", "ConsNo"],
        right_on = ["prov_id", "cons_no"]
    ).merge(const_vote)\
    .drop(columns = [
        "ProvID", "ConsID",
        "GoodBallot1",
        "BadBallot1", "NocastBallot1",
        "TotalUnit", "ConsNo",
        "cons_no"
    ])\
    .merge(party[["id", "party_no"]], left_on = "PartyID", right_on = "id", how = "left")\
    .rename(columns = {"PartyID": "party_id"})
const_data["prov_id"] = const_data["cons_id"].str.split("_").str[0]

party_list_data = party_list_vote.merge(const_district_1)\
    .merge(
        prov_ref[["cons_id", "cons_no", "prov_id"]],
        left_on = ["ProvID", "ConsNo"],
        right_on = ["prov_id", "cons_no"]
    ).drop(
        columns = [
            "ProvID", "ConsID",
            "PartyNumber", "ConsNo", "cons_no"
        ]
    ).rename(
        columns = {
            "PartyID": "party_id",
            "PartyVote": "party_vote"
        }
    )
const_data

,TotalVoter1,VoterAppear1,UnitFinish,cons_id,prov_id,party_id,ApplyNo,Votes,id,party_no
0,130934,86452,177,BKK_1,BKK,701,10,0,701,26.0
1,130934,86452,177,BKK_1,BKK,705,8,4789,705,29.0
2,130934,86452,177,BKK_1,BKK,706,9,2,706,14.0
3,130934,86452,177,BKK_1,BKK,709,6,114,709,7.0
4,130934,86452,177,BKK_1,BKK,719,13,4,719,25.0
...,...,...,...,...,...,...,...,...,...,...
4820,127207,104902,226,NWT_5,NWT,740,4,2234,740,11.0
4821,127207,104902,226,NWT_5,NWT,742,9,243,742,61.0
4822,127207,104902,226,NWT_5,NWT,743,5,761,743,37.0
4823,127207,104902,226,NWT_5,NWT,762,1,4153,762,32.0


## edit

In [18]:
party_list_vote

,ProvID,ConsID,PartyNumber,PartyID,PartyVote
0,10,201,1,778,0
1,10,201,2,712,532
2,10,201,3,747,0
3,10,201,4,776,0
4,10,201,5,761,1445
...,...,...,...,...,...
26795,96,375,63,702,0
26796,96,375,64,729,0
26797,96,375,65,777,0
26798,96,375,66,722,30


In [74]:
stats_data_csv

,prov_id,cons_no,turn_out,counted_vote_stations,total_vote_stations,pause_report
0,10,1,101249,226,238,False
1,10,2,113929,217,228,True
2,10,3,88970,202,213,False
3,10,4,103174,183,193,False
4,10,5,95289,193,203,True
...,...,...,...,...,...,...
395,96,1,110833,200,210,True
396,96,2,80998,160,168,True
397,96,3,79406,186,196,False
398,96,4,90507,191,201,True


In [14]:
stats_data = pd.read_csv("https://ectdatastorage.blob.core.windows.net/data/db_input/ect_static.csv")
const_data = pd.read_csv("https://ectdatastorage.blob.core.windows.net/data/db_output/csv/candidatevotes.csv")
party_list_data = pd.read_csv("https://ectdatastorage.blob.core.windows.net/data/db_output/csv/partyvotes.csv")
stats_data

,ProvID,Province,ConsNo,TotalVoter,VoterAppear,TotalUnit,UnitFinish,timestamp
0,10,กรุงเทพมหานคร,1,38875.0,43358,292,292,2023-05-05T14:05:02
1,10,กรุงเทพมหานคร,1,38875.0,43358,292,292,2023-05-05T14:00:06
2,10,กรุงเทพมหานคร,1,38875.0,43358,292,292,2023-05-05T14:10:04
3,10,กรุงเทพมหานคร,1,38875.0,43358,292,292,2023-05-05T13:50:03
4,10,กรุงเทพมหานคร,1,38875.0,43358,292,292,2023-05-05T13:55:04
...,...,...,...,...,...,...,...,...
1378,96,นราธิวาส,1,10000.0,8080,69,12,2023-05-05T14:30:05
1379,96,นราธิวาส,2,13600.0,14200,169,18,2023-05-05T14:30:05
1380,96,นราธิวาส,3,37050.0,11501,137,21,2023-05-05T14:30:05
1381,96,นราธิวาส,4,10601.0,20801,155,27,2023-05-05T14:30:05


In [19]:
from datetime import datetime
import pandas as pd
import json
import numpy as np

# # for reference
# static = pd.read_csv("https://ectdatastorage.blob.core.windows.net/data/db_input/ect_static.csv")
# party_votes = pd.read_csv("https://ectdatastorage.blob.core.windows.net/data/db_input/ect_partyvotes.csv")
# cons_votes = pd.read_csv("https://ectdatastorage.blob.core.windows.net/data/db_input/ect_candidatevotes.csv")

# print("successfully loaded!")

# prov = pd.DataFrame(
#         json.load(
#             open(
#                 "info_province.json",
#                 encoding = "utf-8-sig"
#             )
#         )["province"]
#     )

# prov["province_id"] = prov["province_id"].astype(int)

# info_const = pd.read_json("info_constituency.json")\
#     .rename(columns = {"cons_id": "ConsID"})\
#     .merge(
#         prov,
#         on = "prov_id"
#     )

# static["timestamp"] = static["timestamp"].apply(
#         lambda x: pd.Timestamp(x)
#     )
# party_votes["timestamp"] = party_votes["timestamp"].apply(
#         lambda x: pd.Timestamp(x)
#     ).apply(
#         lambda x: str(x)
#     )

# cons_votes["timestamp"] = cons_votes["timestamp"].apply(
#         lambda x: pd.Timestamp(x)
#     ).apply(
#         lambda x: str(x)
#     )

# static_last = static.sort_values("timestamp", ascending = False)
# static_last = static_last.merge(
#         info_const[["province_id", "cons_no", "total_vote_stations_y"]],
#         left_on = ["ProvID", "ConsNo"],
#         right_on = ["province_id", "cons_no"],
#         how = "right"
#     ).fillna(0)\
#     .drop(columns = ["ProvID", "ConsNo"])\
#     .rename(columns = {
#         "province_id": "ProvID",
#         "cons_no": "ConsNo"
#     })

# static_last_time = static_last[
#     (static_last["UnitFinish"] / static_last["total_vote_stations_y"]) <= .95
#     ][["ProvID", "Province",
#        "ConsNo", "TotalVoter",
#        "VoterAppear", "TotalUnit",
#        "UnitFinish", "timestamp"]]

# static_final = static_last_time.drop_duplicates(subset = ["ProvID", "ConsNo"])

# # for keeping more than 95% data when there is no data at the first glance.

# # static_final["VoterAppear"] = static_final.apply(
# #         lambda x: 0\
# #             if x.UnitFinish <= 1\
# #             else x.VoterAppear,
# #         axis = 1
# #     )

# # static_final["UnitFinish"] = static_final.apply(
# #         lambda x: 0\
# #             if x.UnitFinish <= 1\
# #             else x.UnitFinish - 1,
# #         axis = 1
# #     )

# static_final["timestamp"] = static_final["timestamp"].apply(
#         lambda x: str(x)
#     )

# stats_data = static_final.merge(
#         info_const[["province_id", "cons_no"]],
#         left_on = ["ProvID", "ConsNo"],
#         right_on = ["province_id", "cons_no"],
#         how = "right"
#     ).fillna(0)[["ProvID", "ConsNo",
#                 "VoterAppear", "UnitFinish",
#                 "timestamp"]]

# stats_data["PauseReport"] = (
#     (stats_data["timestamp"] != str(static["timestamp"]\
#                                     .sort_values()\
#                                     .tolist()[-1]))\
#     & (stats_data["timestamp"] != '0'))

# party_data = static_final[["ProvID", "ConsNo", "timestamp"]]\
#     .merge(
#         party_votes,
#         on = ["ProvID", "ConsNo", "timestamp"],
#         how = "left"
#     )[["ProvID", "ConsNo",
#        "PartyID", "TotalPartyVotes"]]

# tmp = info_const[["province_id", "cons_no"]].copy()
# tmp["key"] = 1

# tmp_2 = (party := pd.DataFrame(
#             json.load(
#                 open("./info_party_overview.json",
#                 encoding = "utf-8-sig")
#             )
#         ))[party["party_no"] != ""][["id"]]\
#         .astype(int)\
#         .rename(columns = {
#             "id": "PartyID"
#         })

# tmp_2["key"] = 1

# template_party = tmp.merge(
#         tmp_2,
#         on = "key"
#     ).drop(columns = "key")\
#     .rename(columns = {
#         "province_id": "ProvID",
#         "cons_no": "ConsNo"
#     })

# party_data = party_data.merge(
#         template_party,
#         on = ["ProvID", "ConsNo", "PartyID"],
#         how = "right"
#     ).fillna(0)\
#     .merge(
#         stats_data,
#         on = ["ProvID", "ConsNo"]
#     )

# # party_data["TotalPartyVotes"] = party_data.apply(
# #         lambda x: 0\
# #             if (x.UnitFinish == 0)\
# #             else x.TotalPartyVotes,
# #         axis = 1
# #     )
# party_data = party_data[["ProvID", "ConsNo",
#                          "PartyID", "TotalPartyVotes"]]

# cons_data = static_final[["ProvID", "ConsNo", "timestamp"]]\
#     .merge(
#         cons_votes,
#         on = ["ProvID", "ConsNo", "timestamp"]
#     )[["ProvID", "ConsNo",
#        "PartyID", "ApplyNo",
#        "TotalCandVotes", "timestamp"]]

# tmp_cons = pd.DataFrame(
#         json.load(
#             open(
#                 "./info_mp_candidate.json",
#                 encoding = "utf-8-sig"
#             )
#         )
#     )

# tmp_cons[["mp_app_id", "cons_no",
#           "mp_app_no"]] = tmp_cons["mp_app_id"].str.split(
#             "_",
#             expand = True
#         )

# tmp_cons = (tmp_b := (tmp_a := tmp_cons.merge(
#         prov[["prov_id", "province_id"]],
#         left_on = "mp_app_id",
#         right_on = "prov_id"
#     )[["province_id", "cons_no", "mp_app_no", "mp_app_party_id"]])\
#     .astype(int)\
#     .merge(
#         cons_data,
#         left_on = ["province_id", "cons_no", "mp_app_no"],
#         right_on = ["ProvID", "ConsNo", "ApplyNo"],
#         how = "left"
#     ).fillna(0))\
#     .merge(
#         stats_data,
#         on = ["ProvID", "ConsNo"],
#         how = "left"
#     )

# # tmp_cons["TotalCandVotes"] = tmp_cons.apply(
# #         lambda x: 0\
# #             if x.UnitFinish == 0\
# #             else x.TotalCandVotes,
# #         axis = 1
# #     )

# # expected output

# party_list_data = party_data[["ProvID", "ConsNo",
#                          "PartyID", "TotalPartyVotes"]].astype(int)

# const_data = tmp_cons[["province_id", "cons_no",
#                      "mp_app_party_id", "mp_app_no",
#                      "TotalCandVotes"
#                 ]].rename(columns = {
#                         "province_id": "ProvID",
#                         "mp_app_party_id": "PartyID",
#                         "cons_no": "ConsNo",
#                         "mp_app_no": "ApplyNo"
#                     }
#                 ).astype(int)

# stats_data = stats_data[["ProvID", "ConsNo",
#                          "VoterAppear", "UnitFinish",
#                          "PauseReport"]].astype(int)

# stats_data = pd.DataFrame({ "ProvID": [10, 10], "ConsNo": [1, 2], "VoterAppear": [50, 1], "UnitFinish": [10000, 1], "PauseReport": [0,0]})
# const_data = pd.DataFrame({ "ProvID": [10,10], "ConsNo": [1,1], "PartyID": [701, 705], "ApplyNo": [1,2], "TotalCandVotes": [100,1] })
# party_list_data = pd.DataFrame({ "ProvID": [10,10], "ConsNo": [1,1], "PartyID": [701.0, 705.0], "TotalPartyVotes": [0.0, 5.0] })

##############################################################################

stats_data = pd.read_csv("stop_stats.csv")
const_data = pd.read_csv("stop_const.csv")
party_list_data = pd.read_csv("stop_party.csv")

stats_data["PauseReport"] = stats_data["PauseReport"].astype(bool)

stats_data_csv = stats_data.copy()
const_data_csv = const_data.copy()
party_list_data_csv = party_list_data.copy()

cons_json = pd.DataFrame(
    json.load(
        open(
                "./info_constituency.json",
                encoding = "utf-8-sig"
            )
        )
    )

prov_json = pd.DataFrame(
        json.load(
            open(
                "./info_province.json",
                encoding = "utf-8-sig"
            )
        )["province"]
    )
prov_json["province_id"] = prov_json["province_id"].astype(int)

def map_prov(df):
    return df.merge(
            prov_json[["province_id", "prov_id"]].merge(
                cons_json[["prov_id", "cons_no", "cons_id"]],
                on = "prov_id"
            ),
            left_on = ["ProvID", "ConsNo"],
            right_on = ["province_id", "cons_no"]
        ).drop(
            columns = ["ProvID", "ConsNo",
                        "province_id", "cons_no"]
        )

stats_data = map_prov(stats_data)\
        .rename(columns = {
            "VoterAppear": "turn_out",
            "UnitFinish": "counted_vote_stations"
        }
    )

mp_json = pd.DataFrame(
        json.load(
            open(
                "./info_mp_candidate.json",
                encoding = "utf-8-sig"
            )
        )
    )

const_data = map_prov(const_data)
const_data["mp_app_id"] = const_data.apply(
        lambda x:
            x.cons_id +
            "_" +
            str(x.ApplyNo),
        axis = 1
    )
const_data = const_data.rename(columns = {
            "TotalCandVotes": "Votes",
            "PartyID": "party_id"
        }
    )

const_data = const_data.merge(stats_data)

party_list_data = map_prov(party_list_data)\
    .merge(stats_data)\
    .rename(columns = {
            "PartyID": "party_id",
            "TotalPartyVotes": "party_vote"
        }
    )

const_data["rank"] = const_data.groupby(["cons_id"])["Votes"]\
    .rank(ascending = False, method = "max")\
    .astype(int)

const_data["candidates"] = const_data.apply(
        lambda x: dict([
                ("mp_app_id", x.cons_id + "_" + str(x.ApplyNo)),
                ("mp_app_vote", x.Votes),
                ("party_id", x.party_id), 
                ("mp_app_vote_percent", round(
                    (x.Votes / x.turn_out * 100)\
                        if x.turn_out != 0\
                        else 0,
                    5)
                ),
                ("mp_app_rank", x["rank"])
            ]),
        axis = 1
    )

const_data_prov = const_data.sort_values("rank")[
    ["prov_id", "cons_id",
        "turn_out", "counted_vote_stations",
        "candidates", "PauseReport"]
    ].groupby(["prov_id", "cons_id",
                "turn_out", "counted_vote_stations",
                "PauseReport"]
    )["candidates"]\
    .apply(list)\
    .reset_index()

party_list_data["result_party"] = party_list_data.apply(
        lambda x: dict([
                ("party_id", x.party_id),
                ("party_list_vote", x.party_vote),
                ("party_list_vote_percent", round(
                    (x.party_vote / x.turn_out * 100)\
                        if x.turn_out != 0\
                        else 0,
                    5)
                )
            ]),
        axis = 1
    )

const_data_prov = const_data_prov.merge(
        party_list_data.groupby("cons_id")["result_party"]\
        .apply(list)\
        .reset_index()
    )
const_data_prov = const_data_prov.merge(
        cons_json[["cons_id", "total_vote_stations", "registered_vote"]]
    )

const_data_prov["constituencies"] = const_data_prov.apply(
    lambda x: dict([
        ("cons_id", x.cons_id),
        ("turn_out", x.turn_out),
        ("percent_turn_out", round(x.turn_out/x.registered_vote * 100, 5)),
        ("candidates", x.candidates),
        ("result_party", x["result_party"]),
        ("counted_vote_stations", x.counted_vote_stations),
        ("percent_count", round(
            x.counted_vote_stations / x.total_vote_stations * 100,
            5)
        ),
        ("pause_report", x.PauseReport)
        ]),
    axis = 1
)

const_overview = const_data_prov[["prov_id", "turn_out",
                                    "counted_vote_stations", "constituencies", "PauseReport"]]\
    .groupby(["prov_id"])\
    .agg({"turn_out": sum,
          "counted_vote_stations": sum,
          "PauseReport": "all"})\
    .reset_index()

const_constituencies = const_data_prov[["prov_id", "turn_out",
                                        "counted_vote_stations", "constituencies"]]\
    .groupby(["prov_id"])["constituencies"]\
    .apply(list)\
    .reset_index()

const_party_prov = const_data.groupby(["prov_id", "party_id"])["candidates"]\
    .apply(list)\
    .reset_index()

const_party_prov[["votes", "first_mp_app_count"]] = const_data[
    (const_data["turn_out"] != 0) & (const_data["counted_vote_stations"] != 0)
    ].groupby(["prov_id", "party_id"])\
    .agg({
        "Votes": sum,
        "rank": (lambda x: (x==1).sum())
    })\
    .merge(
        const_data.groupby(["prov_id", "party_id"]).sum()["turn_out"].reset_index(),
        how = "right",
        on = ["prov_id", "party_id"])\
    .reset_index()\
    .sort_values("rank")[["Votes", "rank"]]

const_party_prov["party_id"] = const_party_prov["party_id"].astype(int)

const_party_prov["party_list"] = const_party_prov.apply(
    lambda x: dict([
            ("party_id", x.party_id),
            ("candidates", x.candidates)
        ]),
    axis = 1
)

const_party_prov = const_party_prov.merge(
    stats_data.groupby("prov_id")\
    .sum()\
    .reset_index()
)

const_party_prov["party_cons_list"] = const_party_prov.apply(
    lambda x: dict([
            ("party_id", x.party_id),
            ("party_cons_votes", x.votes),
            ("party_cons_votes_percent", round(
                (x.votes / x.turn_out * 100)\
                    if x.turn_out != 0\
                    else 0,
                5)
            ),
            ("first_mp_app_count", x.first_mp_app_count)
        ]),
    axis = 1
)

prov_vote = const_overview.merge(const_constituencies)\
    .merge(
        const_party_prov.drop(columns = "candidates")\
        .groupby("prov_id")\
        .agg({
            "party_list": list,
            "party_cons_list": list
        })\
        .reset_index()
    ).merge(
        prov_json,
        on = "prov_id"
    )

prov_vote["percent_turn_out"] = np.round(
        prov_vote["turn_out"] / prov_vote["total_registered_vote"] * 100,
        decimals = 5
    ).astype(float)
prov_vote["percent_count"] = np.round(
        prov_vote["counted_vote_stations"] / prov_vote["total_vote_stations"] * 100,
        decimals = 5
    ).astype(float)

prov_vote["result_const"] = prov_vote.apply(
        lambda x: dict([
                ("prov_id", x.prov_id),
                ("turn_out", x.turn_out),
                ("counted_vote_stations", x.counted_vote_stations), 
                ("percent_turn_out", x.percent_turn_out),
                ("percent_count", x.percent_count),
                ("party_list", x.party_list),
                ("constituencies", x.constituencies)
            ]),
        axis = 1
    )

party_list_data["prov_id"] = party_list_data["cons_id"].str\
    .split("_")\
    .str[0]

party_list_data["party_vote_percent"] = np.round(
        party_list_data["party_vote"] / party_list_data["turn_out"] * 100,
        decimals = 5
    ).astype(float)

party_list_data_prov = party_list_data.groupby(["prov_id", "party_id"])\
    .sum()\
    .reset_index()

party_list_data_prov["result_party"] = party_list_data_prov.apply(
        lambda x: dict([
                ("party_id", x.party_id),
                ("party_list_vote", x.party_vote), 
                ("party_list_vote_percent", round(
                        (x.party_vote / x.turn_out * 100)\
                        if x.turn_out != 0\
                        else 0,
                    5)
                )
            ]),
        axis = 1
    )
prov_vote = prov_vote.merge(party_list_data_prov.sort_values(
            "party_vote",
            ascending = False
        ).groupby("prov_id")["result_party"]\
        .apply(list)\
        .reset_index()
    )

def concat_dict(x):
    result_party = pd.DataFrame(x.result_party)
    party_cons_list = pd.DataFrame(x.party_cons_list)
    return json.loads(
            result_party.merge(
                party_cons_list,
                how = "right"
            ).to_json(orient = "records")
        )

prov_vote["result_party"] = prov_vote.apply(
        concat_dict,
        axis = 1
    )

party_vote = party_list_data.groupby("party_id")\
    .sum()["party_vote"]\
    .reset_index()

party_vote = party_vote.merge(
        const_data.groupby("party_id")\
        .sum()["Votes"]\
        .reset_index(),
        how = "right"
    ).rename(columns = {
        "Votes": "mp_app_vote"
    })

const_data["TotalVoter1"] = stats_data.sum()["turn_out"]
party_vote["TotalVoter1"] = stats_data.sum()["turn_out"]

party_vote["party_vote_percent"] = np.round(
        party_vote["party_vote"] / party_vote["TotalVoter1"] * 100,
        decimals = 5
    ).astype(float)
party_vote["mp_app_vote_percent"] = np.round(
        party_vote["mp_app_vote"] / party_vote["TotalVoter1"] * 100,
        decimals = 5
    ).astype(float)

party_vote = party_vote.merge(
        const_party_prov.groupby("party_id")["candidates"]\
            .sum()\
            .apply(lambda x: 
                json.loads(
                    mp_json.reset_index()\
                    .merge(pd.DataFrame(x))\
                    .sort_values("index")[pd.DataFrame(x).columns]\
                    .to_json(orient = "records")
                )
            ).reset_index(),
        how = "right"
    ).merge(
        (const_data_tmp := const_data.sort_values("rank")\
            .drop_duplicates(
                subset = ["cons_id"],
                keep = "first"
            ))[const_data_tmp["rank"] == 1]\
            .groupby("party_id")\
            .count()["TotalVoter1"]\
            .reset_index()\
            .rename(columns = {
                "TotalVoter1": "first_mp_app_count"
            }),
        how = "left"
    ).fillna(0)

party_vote["first_mp_app_count"] = party_vote["first_mp_app_count"].astype(int)

prov_vote = prov_vote.rename(columns = {"PauseReport": "pause_report"})

################### JSON part #######################

election_result = dict()
election_result["counted_vote_stations"] = int(prov_vote["counted_vote_stations"].sum())

total_vote_stations_th = json.load(open("./info_province.json"))["total_vote_stations"]
total_reg_voters_th = json.load(open("./info_province.json"))["total_registered_vote"]

election_result["percent_count"] = round(
        election_result["counted_vote_stations"] / total_vote_stations_th * 100,
        5
    )

election_result["result_party"] = json.loads(
        party_vote.drop(columns = ["TotalVoter1"])\
        .to_json(orient = "records")
    )

election_cons = dict()
election_cons["turn_out"] = int(prov_vote["turn_out"].sum())
election_cons["counted_vote_stations"] = int(prov_vote["counted_vote_stations"].sum())

election_cons["percent_turn_out"] = round(
        election_cons["turn_out"] / total_reg_voters_th * 100,
        5
    )
election_cons["percent_count"] = round(
        election_cons["counted_vote_stations"] / total_vote_stations_th * 100,
        5
    )

election_cons["pause_report"] = bool(prov_vote["pause_report"].all())

election_cons["result_province"] = json.loads(
        prov_vote.drop(
        columns = ["party_list", "result_const",
                    "party_cons_list", "province_id",
                    "province", "abbre_thai",
                    "eng"]
        ).to_json(orient = "records")
    )

election_cons["last_update"] = election_result["last_update"] =\
    datetime.now().strftime("%Y-%m-%dT%X+0700")

election_cons["processor"] = election_result["processor"] =\
    "Government Big Data Institute"

election_cons["source"] = election_result["source"] =\
    "สำนักงานคณะกรรมการการเลือกตั้ง"

############################ CSV part ###########################################

stats_data_csv = stats_data_csv.merge(
        prov_json[["province_id", "prov_id"]],
        left_on = "ProvID",
        right_on = "province_id"
    ).merge(
        cons_json,
        left_on = ["prov_id", "ConsNo"],
        right_on = ["prov_id", "cons_no"]
    )[["ProvID", "ConsNo",
        "VoterAppear", "UnitFinish",
        "total_vote_stations", "PauseReport"]]

stats_data_csv = stats_data_csv.rename(columns = {
    "ProvID": "prov_id",
    "ConsNo": "cons_no",
    "VoterAppear": "turn_out",
    "UnitFinish": "counted_vote_stations",
    "PauseReport": "pause_report"
}).sort_values(["prov_id", "cons_no"])

stats_data_csv["pause_report"] = stats_data_csv["pause_report"].astype(bool)

mp_json_csv = mp_json.copy()
mp_json_csv["prov_id"] = mp_json_csv["mp_app_id"].str\
    .split("_").str[0]
mp_json_csv["cons_no"] = mp_json_csv["mp_app_id"].str\
    .split("_").str[1]\
    .astype(int)

const_data_csv = const_data_csv.merge(
        prov_json[["province_id", "prov_id"]],
        left_on = "ProvID",
        right_on = "province_id"
    ).merge(
        mp_json_csv,
        left_on = ["prov_id", "ConsNo", "ApplyNo"],
        right_on = ["prov_id", "cons_no", "mp_app_no"])[["ProvID", "ConsNo",
                                                            "mp_app_no", "mp_app_party_id",
                                                            "mp_app_name", "TotalCandVotes"]]\
    .rename(columns = {
        "ProvID": "prov_id",
        "ConsNo": "cons_no",
        "mp_app_party_id": "party_id",
        "TotalCandVotes": "mp_app_vote"
    }).sort_values(["prov_id", "cons_no", "mp_app_no"])

party_overview = pd.DataFrame(
        json.load(
            open(
                "./party_overview_json.json",
                encoding = "utf-8-sig"
            )
        )
    )

party_overview["id"] = party_overview["id"].astype(int)
party_list_data_csv = party_list_data_csv.merge(
        party_overview,
        left_on = "PartyID",
        right_on = "id"
    )[["ProvID", "ConsNo",
        "PartyID", "party_no",
        "name", "TotalPartyVotes"
    ]].rename(columns = {
        "ProvID": "prov_id",
        "ConsNo": "cons_no",
        "PartyID": "party_id",
        "name":"party_name",
        "TotalPartyVotes": "party_vote"
    }).sort_values(["prov_id", "cons_no", "party_id"])

name = {
    "response": {
        "stats_cons_json": election_cons,
        "stats_party_json": election_result,
        "stats_cons_csv": json.loads(
            stats_data_csv.to_json(orient = "records")
        ),
        "result_cons_csv": json.loads(
            const_data_csv.to_json(orient = "records")
        ),
        "result_party_csv": json.loads(
            party_list_data_csv.to_json(orient = "records")
        )
    }
}

with open("stats_cons_stop.json", mode = "w") as f:
    f.write(json.dumps(election_cons, ensure_ascii = False))
with open("stats_party_stop.json", mode = "w") as f:
    f.write(json.dumps(election_result, ensure_ascii = False))

In [10]:
prov_vote

,prov_id,turn_out,counted_vote_stations,PauseReport,constituencies,party_list,party_cons_list,province_id,province,abbre_thai,eng,total_vote_stations,total_registered_vote,percent_turn_out,percent_count,result_const,result_party
0,ACR,0,0,False,"[{'cons_id': 'ACR_1', 'turn_out': 0, 'percent_...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': nan, 'p...",37,อำนาจเจริญ,อจ,AMNAT CHAROEN,641,301354,0.00000,0.00000,"{'prov_id': 'ACR', 'turn_out': 0, 'counted_vot...","[{'party_id': 701, 'party_list_vote': 0, 'part..."
1,ATG,0,0,False,"[{'cons_id': 'ATG_1', 'turn_out': 0, 'percent_...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': nan, 'p...",15,อ่างทอง,อท,ANG THONG,492,224656,0.00000,0.00000,"{'prov_id': 'ATG', 'turn_out': 0, 'counted_vot...","[{'party_id': 701, 'party_list_vote': 0, 'part..."
2,AYA,1100,100,False,"[{'cons_id': 'AYA_1', 'turn_out': 1000, 'perce...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': 1000.0,...",14,พระนครศรีอยุธยา,อย,PHRA NAKHON SI AYUTTHAYA,1060,664431,0.16556,9.43396,"{'prov_id': 'AYA', 'turn_out': 1100, 'counted_...","[{'party_id': 701, 'party_list_vote': 0, 'part..."
3,BKK,885424,1373,False,"[{'cons_id': 'BKK_1', 'turn_out': 0, 'percent_...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': 50110.0...",10,กรุงเทพมหานคร,กทม,BANGKOK,6360,4479155,19.76766,21.58805,"{'prov_id': 'BKK', 'turn_out': 885424, 'counte...","[{'party_id': 701, 'party_list_vote': 0, 'part..."
4,BKN,0,0,False,"[{'cons_id': 'BKN_1', 'turn_out': 0, 'percent_...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': nan, 'p...",38,บึงกาฬ,บก,BUENG KAN,652,327745,0.00000,0.00000,"{'prov_id': 'BKN', 'turn_out': 0, 'counted_vot...","[{'party_id': 701, 'party_list_vote': 0, 'part..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,UDN,15944,86,False,"[{'cons_id': 'UDN_1', 'turn_out': 0, 'percent_...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': 1402.0,...",41,อุดรธานี,อด,UDON THANI,2252,1248824,1.27672,3.81883,"{'prov_id': 'UDN', 'turn_out': 15944, 'counted...","[{'party_id': 701, 'party_list_vote': 112, 'pa..."
73,UTI,0,0,False,"[{'cons_id': 'UTI_1', 'turn_out': 0, 'percent_...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': nan, 'p...",61,อุทัยธานี,อน,UTHAI THANI,617,260983,0.00000,0.00000,"{'prov_id': 'UTI', 'turn_out': 0, 'counted_vot...","[{'party_id': 701, 'party_list_vote': 0, 'part..."
74,UTT,0,0,False,"[{'cons_id': 'UTT_1', 'turn_out': 0, 'percent_...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': nan, 'p...",53,อุตรดิตถ์,อต,UTTARADIT,732,368323,0.00000,0.00000,"{'prov_id': 'UTT', 'turn_out': 0, 'counted_vot...","[{'party_id': 701, 'party_list_vote': 0, 'part..."
75,YLA,0,0,False,"[{'cons_id': 'YLA_1', 'turn_out': 0, 'percent_...","[{'party_id': 701, 'candidates': [{'mp_app_id'...","[{'party_id': 701, 'party_cons_votes': nan, 'p...",95,ยะลา,ยล,YALA,596,382271,0.00000,0.00000,"{'prov_id': 'YLA', 'turn_out': 0, 'counted_vot...","[{'party_id': 701, 'party_list_vote': 0, 'part..."


In [24]:
from azure.storage.blob import BlobServiceClient

block_blob_service = BlobServiceClient(
        account_url = "https://ectdatastorage.blob.core.windows.net/",
        credential = "idLGv4kDiLK53tcFaCR7G0D8Dc0bXiNmAceJBvm+vpGG9UjK1p15HsEq3hwQjjTeawOdC1hBa5Vw+AStxZf5LQ==" # "Joxc3+R3UVkTLYMOoNafS3g+scOPIgBOvfq5rHM5EyRjDraI3mCxMkHm0MvwZ+fsslzWEg0kdFS2+AStaPh3ag=="
)

with open(f"./{(filename := 'stats_cons.json')}", "wb") as my_blob:
    stream = block_blob_service.get_blob_client("data", f"stats/{filename}").download_blob()
    data = stream.readall()
    my_blob.write(data)

In [53]:
const_data.groupby(["prov_id", "party_id"]).sum()["turn_out"].reset_index()

,prov_id,party_id,turn_out
0,ACR,701,0
1,ACR,705,0
2,ACR,706,0
3,ACR,709,0
4,ACR,712,0
...,...,...,...
1144,YST,743,0
1145,YST,755,0
1146,YST,762,0
1147,YST,763,0


In [10]:
const_data.groupby(["prov_id", "party_id"])\
    .agg({
        "Votes": sum,
        "rank": (lambda x: (x==1).sum())
    }).sort_values("rank")

Votes  rank
prov_id party_id             
ACR     701        9092     0
RBR     701       20574     0
        705       20574     0
        706       12375     0
        709       20574     0
...                 ...   ...
MDH     743        8351     0
        742        8351     0
        731        4140     0
        769        8351     0
YST     769       13067     0

[1149 rows x 2 columns]